In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# library
import seaborn as sns

#Importamos dependencias
import statsmodels.api as sm
import statsmodels.formula.api as sfm

from matplotlib.widgets import Slider, Button, RadioButtons
from scipy import interp
from scipy.optimize import fsolve
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from statsmodels.formula.api import ols

ModuleNotFoundError: No module named 'plotly'

In [ ]:
!ls

> ¿Está la adherencia relacionada con los hábitos?

In [ ]:
adh_df = pd.read_csv('Adher.csv')

In [ ]:
adh_df.info()

In [ ]:
adh_df.describe(include = 'all').T

In [ ]:
adh_df.head() #adh_df[['id_patient', 'survey_date', 'quantitative_result']]

In [ ]:
adh_df.dtypes

In [ ]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=['NO APLICA', 
                                        '<30%',
                                        '30-65%',
                                        '64-84%',
                                        '85-94%',                                       
                                        '95-100%'], ordered=True)
adh_df['quantitative_result'] = adh_df['quantitative_result'].astype(cat_type)

In [ ]:
adh_df['quantitative_result'].cat.codes.head()

In [ ]:
adh_df['id_patient'].value_counts().head()

In [ ]:
df_0 = adh_df[adh_df['id_patient'] == 1769865].copy()
df_0.sort_values('quantitative_result').head()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_0.sort_values('quantitative_result'), 
                 x="survey_date", 
                 y="quantitative_result", 
                 color="qualitative_result")
fig.update_layout(yaxis_type='category',
                  title_text='Bar chart with categorical axes')
fig.show()

In [ ]:
#Dataset de hábitos Habitos.xlsx
hab_df = pd.read_csv('habits.csv')

In [ ]:
hab_df.head().columns

In [ ]:
df_1 = hab_df[['id_patient', 'registration_date']].rename(columns = {'registration_date':'r_date'}).copy()
df_1['quantitative_result'] = np.nan #qualitative_result
df_1['qualitative_result'] = np.nan
df_1['source'] = 'habits'
print(df_1.shape)
df_1.head()

In [ ]:
df_2 = adh_df[['id_patient', 'survey_date', 'quantitative_result', 'qualitative_result']].rename(columns = {'survey_date':'r_date'}).copy()
df_2['source'] = 'adherence'
print(df_2.shape)
df_2.head()

In [ ]:
df_3 = df_1.append(df_2)
print(df_3.shape)
df_3['id_patient'].value_counts().head(20)

In [ ]:
df_3[df_3['id_patient'] == 1769865].sort_values('r_date')

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_3[df_3['id_patient'] == 1769865].sort_values('quantitative_result'), x="r_date", y="quantitative_result")
fig.update_layout(yaxis_type='category',
                  title_text='Bar chart with categorical axes')
fig.show()

In [ ]:
df_3.head()

In [ ]:
df_3.dtypes

In [ ]:
cat_type = CategoricalDtype(categories=['NO APLICA', 
                                        '<30%',
                                        '30-65%',
                                        '64-84%',
                                        '85-94%',                                       
                                        '95-100%'], ordered=True)
df_3['quantitative_result'] = df_3['quantitative_result'].astype(cat_type)

In [ ]:
#qualitative_result
df_3['qualitative_result'].value_counts()

cat_type = CategoricalDtype(categories=['si', 
                                        'no'], ordered=True)
df_3['qualitative_result'] = df_3['qualitative_result'].astype(cat_type)

In [ ]:
def fill_nas(g):
    if sum(g['quantitative_result'].notna()) > 0:
        #Backward
        g['quantitative_result'] = g['quantitative_result'].fillna(method='bfill')
        g['qualitative_result'] = g['qualitative_result'].fillna(method='bfill')
        #Forward
        g['quantitative_result'] = g['quantitative_result'].interpolate(method='pad')
        g['qualitative_result'] = g['qualitative_result'].interpolate(method='pad')
    if sum(g['quantitative_result'].isna()) > 0:
        print(g)
    return g

#g['quantitative_result'].interpolate(method='pad', limit_direction = 'backward')
print(df_3.sort_values(by = ['id_patient', 'r_date']).groupby('id_patient').apply(lambda x:x).info())
df_4 = df_3.sort_values(by = ['id_patient', 'r_date']).groupby('id_patient').apply(lambda x:fill_nas(x))
df_4

In [ ]:
user = 1764814
fig = px.scatter(df_4[df_4['id_patient'] == user].sort_values('quantitative_result'), 
                 x="r_date", 
                 y="quantitative_result",
                color="source")
fig.update_layout(yaxis_type='category',
                  title_text='Bar chart with categorical axes')
fig.show()

print(df_4[df_4['id_patient'] == user])

In [ ]:
df_4[df_4['source']=='habits'].head()

In [ ]:
df_4.columns

In [ ]:
df_4[df_4['source']=='habits'].groupby(by = ['id_patient', 'r_date'], as_index = False)

In [ ]:
hab_df.groupby(by = ['id_patient','registration_date'] ).apply(lambda x: )

In [ ]:
hab_df.drop_duplicates().shape

In [ ]:
hab_df.head()

In [ ]:
hab_df['habit_one'].value_counts().plot.bar()

In [ ]:
df_5 = hab_df.join(df_4[df_4['source']=='habits'][['quantitative_result', 'qualitative_result']])
df_5.head()

In [ ]:
selected_features = ['habit_one', 'habit_two']

df_5['registration_date'] = pd.to_datetime(df_5['registration_date'])

for column in selected_features:
  df_5[column] = df_5[column].astype('category')

In [ ]:
df_5.dtypes

In [ ]:
#%%time
vars_to_see_0 = ['habit_one', 
                 'habit_two',
                 'quantitative_result'                 
               ]
df = df_5[df_5['qualitative_result'].notna()]
for c in vars_to_see_0:
    vars_to_see = [c]
    dfs = []
    for column in vars_to_see:
      dfs.append(pd.get_dummies(df[column], prefix=column, drop_first=True))

    df_log_x = pd.concat(dfs, axis=1)
    df_log_x['Intercept'] = 1

    logit_full1 = sm.Logit(df['qualitative_result'].cat.codes, df_log_x)
    logit_full1_res = logit_full1.fit()
    predict_withvs = logit_full1_res.predict(df_log_x)
    print(logit_full1_res.summary())

    #%matplotlib inline

    #Next we plot the ROC curve:
    roc_vs = roc_curve(df['qualitative_result'].astype('category').cat.codes, predict_withvs)
    auc_vs = auc( roc_vs[0], roc_vs[1] )

    plt.figure()
    line_width = 2
    plt.plot(roc_vs[0], roc_vs[1], color='darkorange', lw=line_width,
             label=f"ROC based on training data (AUC = {auc_vs:0.2f})")
    plt.plot([0, 1], [0, 1], color='navy', lw=line_width, linestyle='--', label='Random guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Classification of Defaulters')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
%%time
vars_to_see = ['habit_one', 
                 'habit_two',
                 #'quantitative_result'                 
               ]

df = df_5[df_5['qualitative_result'].notna()]

dfs = []
for column in vars_to_see:
  dfs.append(pd.get_dummies(df[column], prefix=column, drop_first=True))

df_log_x = pd.concat(dfs, axis=1)

In [ ]:
temp_df = df_log_x.corr().stack().reset_index().sort_values(by = 0, ascending = False)
temp_df[temp_df['level_0'] != temp_df['level_1']].head(10)

In [ ]:
df_log_x.drop(['habit_one_ESTADO ANÍMICO', 'habit_one_ALIMENTACION','habit_one_EJERCICIO'], axis='columns', inplace=True)

#Configura intercepto
df_log_x['Intercept'] = 1

logit_full1 = sm.Logit(df['qualitative_result'].astype('category').cat.codes, df_log_x)
logit_full1_res = logit_full1.fit()
predict_withvs = logit_full1_res.predict(df_log_x)
print(logit_full1_res.summary())

#%matplotlib inline

#Next we plot the ROC curve:
roc_vs = roc_curve(df['qualitative_result'].astype('category').cat.codes, predict_withvs)
auc_vs = auc( roc_vs[0], roc_vs[1] )

plt.figure()
line_width = 2
plt.plot(roc_vs[0], roc_vs[1], color='darkorange', lw=line_width,
         label=f"ROC based on training data (AUC = {auc_vs:0.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=line_width, linestyle='--', label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Classification of Defaulters')
plt.legend(loc='lower right')
plt.show()

In [ ]:
df_temp = pd.read_excel('Habitos.xlsx')

In [ ]:
hab_df.head()

In [ ]:
temp_df = hab_df[['id_patient', 
        'registration_date', 
        'habit_one',
       'habit_two']].groupby( by = ['id_patient', 
                                    'registration_date',
                                    'habit_one'],
                            as_index = False).first().sort_values( by = 'habit_two', 
                                                                      ascending = False)

temp_df.drop_duplicates(inplace=True)
temp_df = temp_df.loc[~temp_df.index.duplicated(keep='first')]

temp_df.pivot(index=['id_patient', 'registration_date'], columns=['habit_one'], values=['habit_two'])


In [ ]:
hab_df.pivot(index=['id_patient', 'registration_date'], columns='habit_one', values='habit_two')